# Assignment 2: Optimization 1 (Solution)

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

These notes were developed by Roya Firoozi and Francesco Borrelli at UC Berkeley. They are protected by U.S. copyright law and by University policy (https://copyright.universityofcalifornia.edu/resources/ownership-course-materials.html).

If you are enrolled in ME C231A/EE C220B you may take notes and make copies of course materials for your own use. You may also share those materials with another student who is registered and enrolled in this course, and with DSP.

You may not reproduce, distribute or display (post/upload) (Links to an external site.) lecture notes or recordings or course materials in any other way — whether or not a fee is charged — without my express written consent. You also may not allow others to do so. If you do so, you may be subject to student conduct proceedings under the Links to an external site.Berkeley Code of Student Conduct, including Sections 102.23 and 102.25.

***

# <font color=blue> Question 1. Linear programming, by hand </font>

# <font color=blue> Question 2. Quadratic programming, by hand </font>

The written solutions for questions 1, 2 can be found in the attached pdf file.

# <font color=blue> Question 3. Approximately verifying optimality, by hand </font>

 $\textbf{Part (a)}$

 The cost at $z^*$ is 0.25.

 Evaluate the cost at for example (z1=0.6, z2=0.4, z3=0), also at (z1=0.46, z2=0.55, z3=0) and some other points to see the given point is a local minimum. The cost at nearby points is greater than the cost at $z^*$, the local minimum.

$\textbf{Part (b)}$ This optimization problem has a quadratic cost and linear constraints, so it is a convex quadratic program (QP). For a convex optimization problem, every locally optimal solution is globally optimal. Thus, $z^*$ is also a global minimum.

***

# <font color=blue> 4. Using $\texttt{linprog}$ to solve LPs: </font>

In [ ]:
import numpy as np
import scipy as cp
from scipy.optimize import linprog

Read about the features and syntaxes of $\texttt{scipy.optimize.linprog}$ package here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html. Then use it to solve the following problem.
Let $x,y,z\in\mathbb{R}$.
\begin{align*}
\min_{x,y,z}\ &\ x+y+z\\
\text{subject to}  &\ 2\le x\\
&-1\le y\\
&-3\le z\\
&x-y+z\ge 4
\end{align*}

To get you started, here is one form of a solution.

In [ ]:
f = np.ones((3,))   # 1D array
A = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [-1, 1, -1]])   # 2D array
b = np.array([-2,1,3,-4])   # 1D array
# default bound is (0, None), so here we need to specify bound as (None, None),
# since all the bounds all already considered in the inequality constraints
opt1 = cp.optimize.linprog(c=f, A_ub=A, b_ub=b, bounds=(None, None), method='simplex')
print('x*=', opt1.x)
print('J*=', opt1.fun)

x*= [ 6. -1. -3.]
J*= 2.0


You can also use the elementwise bounds to solve the same problem in a slightly different way.

In [ ]:
f = np.ones((3,))
A = np.array([[-1, 1, -1]])
b = np.array([-4])
x_bounds = (2, None)
y_bounds = (-1, None)
z_bounds = (-3, None)
# opt2 = cp.optimize.linprog(c=f, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds, z_bounds], method='simplex')
opt2 = cp.optimize.linprog(c=f, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds, z_bounds])
print('x*=', opt2.x)
print('J*=', opt2.fun)

x*= [ 4.00000001 -1.         -0.99999999]
J*= 2.0000000134883065


You might get different values of $x, y, z$ but the same cost for each solution. Why?

The solver provides one optimal solution among many optimal solutions. Hence, depnding on the choice of solver the optimizer can be different, but the optimal value is the same.

***

# <font color=blue> 5. Using $\texttt{cvxopt}$ to solve QPs: </font>

Read about the features and syntaxes of $\texttt{cvxopt}$ online.
Constrained least-squares problems are useful when your decision parameters are known to reside in a constrained set. Here, we show how to solve a constrained least-squares as a quadratic program. The constrained least-squares problem is of the form

\begin{align}
& \min_{x} \|Ax-b\|_2^2\\
& \text{subject to} \ l_i \le x_i \le u_i
\end{align}

Observe that

\begin{align}
\|Ax-b\|_2^2 = (x^TA^T-b^T)(Ax-b)=x^TA^TAx-2b^TAx+b^Tb
\end{align}

We can drop the constant term $b^Tb$ in the optimization program and add it back later. The constrained least-squares problem becomes

\begin{align}
& \min_{x} \  \frac{1}{2}x^T\left(2A^TA \right)x+ (-2A^T b)^Tx\\
& \text{subject to } \ l_i\le x_i\le u_i,
\end{align}

which is a quadratic program. For the following $A$ and $b$ for $X \in \mathbb{R}^5$:

In [ ]:
n = 5 # dimenstion of x
# A = np.random.randn(n, n)
# b = np.random.randn(n)
A = np.array([[-0.54, -1.81,  0.25, -0.46],   # A and b matrices are generated using np.random.randn and kept fixed for the homework.
 [-0.38,  0.37, -2.48, -0.68],
 [-1.31,  0.74, -1.57,  0.28],
 [-0.31, -0.02,  0.75,  0.20]])
b = np.array([ 0.40, -2.45, -0.23, 0.98])
l_i = -0.5
u_i = 0.5

use the $\texttt{cvxopt}$ to solve the constrained least-squares problem.

In [ ]:
import cvxopt
from cvxopt import matrix, solvers

P = 2*A.T@A
q = -2*A.T@b
G = np.concatenate([np.eye(n), -np.eye(n)], axis=0)
h = np.concatenate([u_i*np.ones((n,)), -l_i*np.ones((n,))], axis=0)

P = cvxopt.matrix(P, tc='d')
q = cvxopt.matrix(q, tc='d')
G = cvxopt.matrix(G, tc='d')
h = cvxopt.matrix(h, tc='d')
sol = cvxopt.solvers.qp(P,q,G,h)

print('x*=', sol['x'])
print('p*=', sol['primal objective'] + b.T@b)

     pcost       dcost       gap    pres   dres
 0: -6.7988e+00 -1.2012e+01  2e+01  2e+00  1e-16
 1: -6.3888e+00 -9.0588e+00  4e+00  3e-01  4e-17
 2: -6.0963e+00 -6.3893e+00  3e-01  3e-03  6e-16
 3: -6.1599e+00 -6.1690e+00  9e-03  4e-05  1e-16
 4: -6.1606e+00 -6.1607e+00  1e-04  4e-07  4e-17
 5: -6.1606e+00 -6.1606e+00  1e-06  4e-09  1e-16
Optimal solution found.
x*= [-3.00e-01]
[-2.47e-01]
[ 5.00e-01]
[ 5.00e-01]

p*= 1.0152149348586423


Compare the result to analytical solution with projection:
standard least-squares, followed by variable clipping to enforce constraints after-the-fact.

In [ ]:
xstar = (np.linalg.inv((A.T @ A))) @ (A.T.dot(b))
xstar[xstar > u_i] = u_i  # projection
xstar[xstar < l_i] = l_i  # projection
print('x_analytical:', xstar)
print('x_cvxopt:', sol['x'])
print(np.linalg.norm(A @ np.squeeze(sol['x']) - b))
print(np.linalg.norm(A @ xstar - b))

x_analytical: [-0.5        -0.20267715  0.5         0.5       ]
x_cvxopt: [-3.00e-01]
[-2.47e-01]
[ 5.00e-01]
[ 5.00e-01]

1.0075787487132912
1.0561293728400345


Finding the analytical solution and then projecting the solution on the feasible set results in approximated solution. The exact solution must be computed by calling a solver such as $\texttt{cvxopt}$.

***